In [1]:
import torch
from torch.autograd import Variable
from mpc import mpc
from mpc.mpc import QuadCost, LinDx

In [2]:
torch.manual_seed(0)

n_batch, n_state, n_ctrl, T = 2, 3, 4, 5
n_sc = n_state + n_ctrl

# Randomly initialize a PSD quadratic cost and linear dynamics.
C = torch.randn(T*n_batch, n_sc, n_sc)
C = torch.bmm(C, C.transpose(1, 2)).view(T, n_batch, n_sc, n_sc)
c = torch.randn(T, n_batch, n_sc)

alpha = 0.2
R = (torch.eye(n_state)+alpha*torch.randn(n_state, n_state)).repeat(T, n_batch, 1, 1)
S = torch.randn(T, n_batch, n_state, n_ctrl)
F = torch.cat((R, S), dim=3)

# The initial state.
x_init = torch.randn(n_batch, n_state)

# The upper and lower control bounds.
u_lower = -torch.rand(T, n_batch, n_ctrl)
u_upper = torch.rand(T, n_batch, n_ctrl)

x_lqr, u_lqr, objs_lqr = mpc.MPC(
    n_state=n_state,
    n_ctrl=n_ctrl,
    T=T,
    u_lower=u_lower, 
    u_upper=u_upper,
    lqr_iter=20,
    verbose=1,
    backprop=False,
    exit_unconverged=False,
)(x_init, QuadCost(C, c), LinDx(F))

Initial mean(cost): 3.9041e+01
| iter | mean(cost) | ||full_du||_max | mean(alphas) | total_qp_iters |
| 0 | 6.6806e+00 | 1.94e+00 | 1.00e+00 | 16 |
| 1 | 6.4417e+00 | 2.39e+00 | 6.00e-01 | 15 |
| 2 | 4.5778e+00 | 6.19e-01 | 1.00e+00 | 13 |
| 3 | 4.4537e+00 | 2.90e-01 | 1.00e+00 | 13 |
| 4 | 4.4527e+00 | 6.44e-02 | 1.00e+00 | 11 |
| 5 | 4.4527e+00 | 1.92e-07 | 1.00e+00 | 6 |
| 6 | 4.4527e+00 | 1.96e-07 | 1.00e+00 | 8 |
| 7 | 4.4527e+00 | 5.30e-07 | 5.20e-01 | 10 |
| 8 | 4.4527e+00 | 7.19e-08 | 6.00e-01 | 8 |
